In [2]:
from selenium import webdriver
import pandas as pd
import numpy as np
from selenium.webdriver.common.by import By
import random
from datetime import datetime, timedelta
import pickle
from fuzzywuzzy import fuzz, process
from fractions import Fraction
import time
import collections
import concurrent.futures
from collections import defaultdict
import json
import calendar

In [58]:
def get_stats(url):
    driver = webdriver.Chrome()
    try:
        driver.implicitly_wait(random.randint(0, 6))
        driver.get(url)
        ht = driver.find_elements(By.XPATH, '//span[@class="css-8ngw4i-bottomRow"]')[1].text
        button = driver.find_elements(By.XPATH, '(//button[@class="css-12l9xvx-ExpandButton e7i6a0l0"])[2]')[0]
        div1 = driver.find_elements(By.XPATH, '//li[@class="css-radwzz-Stat e683amr6"]')
        t = {k[1]: [k[0], k[2]] for k in [i.split('\n') for i in [x for x in [d.text for d in div1]
              if x not in ['Shots','Passes','Expected goals (xG)','Discipline','Defence','Duels']]]}
        score = driver.find_element(By.XPATH, '//span[@class="css-bw7eig-topRow"]').get_attribute('textContent').split(' ')
        t['score'] = [score[0], score[2]]
        posession = driver.find_element(By.XPATH, '//div[@class="css-7s52se-PossessionWheel e683amr3"]').text.split('\n')
        t['posession'] = [posession[1].replace('%',''), posession[2].replace('%','')]
        league = driver.find_element(By.XPATH, '//span[@class="css-784syn-LeagueName e1fpx9th0"]').text
        game_stats = {}
        for name, stats in t.items():
            if '(' in ''.join(stats):
                s = stats[0].split(' ') + stats[1].split(' ')
                if len(s) == 4:
                    s = [i.replace('(', '').replace(')','').replace('%','') for i in s]
                    game_stats[name] = [s[0], s[2]]
                    game_stats[name + ' percentage'] = [int(s[1])/100, int(s[3])/100]
                else:
                    s = [i for i in s if '%' not in i]
                    game_stats[name] = s
            else:
                game_stats[name] = stats
        game_stats = {k: float(v[0]) - float(v[1]) for k, v in game_stats.items()}
        game_stats['match_name'] = url.split('/')[-1]
        game_stats['half_time'] = ht
        game_stats['league'] = league
        games_stats['url'] = url
    except:
        game_stats = []
    return game_stats

In [56]:
get_stats('https://www.fotmob.com/match/3358698/matchfacts/kilmarnock-vs-hibernian')

{'Expected goals (xG)': -0.53,
 'Total shots': -3.0,
 'Big chances': -1.0,
 'Big chances missed': 0.0,
 'Accurate passes': 82.0,
 'Accurate passes percentage': 0.08000000000000007,
 'Fouls committed': -3.0,
 'Offsides': 2.0,
 'Corners': -5.0,
 'Shots off target': 2.0,
 'Shots on target': -2.0,
 'Blocked shots': -3.0,
 'Hit woodwork': 0.0,
 'Shots inside box': 2.0,
 'Shots outside box': -5.0,
 'xG first half': -0.20999999999999996,
 'xG second half': -0.32000000000000006,
 'xG open play': -0.35,
 'xG set play': 0.61,
 'xG penalty': -0.79,
 'xG on target (xGOT)': -1.3,
 'Passes': 71.0,
 'Own half': 31.0,
 'Opposition half': 51.0,
 'Accurate long balls': -1.0,
 'Accurate long balls percentage': 0.020000000000000018,
 'Accurate crosses': 5.0,
 'Accurate crosses percentage': 0.10000000000000003,
 'Throws': -1.0,
 'Yellow cards': 1.0,
 'Red cards': 0.0,
 'Tackles won': 0.0,
 'Tackles won percentage': 0.0,
 'Interceptions': -4.0,
 'Blocks': 0.0,
 'Clearances': -11.0,
 'Keeper saves': 1.0,
 'D

In [9]:
xg_leagues = {'UNITED STATES - MAJOR LEAGUE SOCCER':'world/usa/mls','ENGLAND - CHAMPIONSHIP':'english/championship',
              'ENGLAND - PREMIER LEAGUE': 'english/premier-league', 'GERMANY - BUNDESLIGA': 'germany/bundesliga',
              'FRANCE - LIGUE 1': 'france/ligue-1','AUSTRIA - BUNDESLIGA': 'austria/bundesliga',
              'PORTUGAL - LIGA PORTUGAL':'portugal/primeira-liga', 'BELGIUM - FIRST DIVISION A':'belgium/jupiler-pro-league',
              'SWITZERLAND - SUPER LEAGUE':'switzerland/super-league','SPAIN - LALIGA':'spain/la-liga-primera',
             'DENMARK - SUPERLIGAEN':'denmark/superligaen', 'MEXICO - LIGA MX APERTURA':'world/mexico/liga-mx',
             'NETHERLANDS - EREDIVISIE':'netherlands/eredivisie', 'SCOTLAND - PREMIERSHIP': 'scottish/premiership'}

In [17]:
def fotmob_game_data(url):
    try:
        driver = webdriver.Chrome()
        driver.implicitly_wait(3)
        driver.get(url)
        divs = driver.find_elements(By.XPATH, '//div[@class="css-aoqfcq-Group e7pc1841"]')
        links = []
        for count, bucket in enumerate(divs):
            league_name = bucket.find_elements(By.XPATH, '//a[@class="css-vfisw6-GroupTitleLink e7pc1842"]')[count].text
            if league_name not in xg_leagues.keys():
                continue
            url = bucket.find_elements(By.TAG_NAME, 'a')
            urls = [u.get_attribute('href') for u in url]
            urls = urls[1:]
            links.extend(urls)
    except:
        links = []
    return links

In [11]:
d1 = datetime(2020, 1, 1)
d2 = datetime(2022, 6, 10)
days = [d1 + timedelta(days=x) for x in range((d2-d1).days + 1)]
days = [d.strftime('%Y%m%d') for d in days]

In [12]:
links = [f'https://www.fotmob.com/?date={day}' for day in days]

In [28]:
data = []
for i in range(0, len(links), 5):
    chunk = links[i:i+5]
    with concurrent.futures.ThreadPoolExecutor() as executor:
        results = executor.map(fotmob_game_data, chunk)
    for result in results:
        data.append(result)
    print(f'{len(links) - i} links to go')
    time.sleep(30)

892 links to go
887 links to go
882 links to go
877 links to go
872 links to go
867 links to go
862 links to go
857 links to go
852 links to go
847 links to go
842 links to go
837 links to go
832 links to go
827 links to go
822 links to go
817 links to go
812 links to go
807 links to go
802 links to go
797 links to go
792 links to go
787 links to go
782 links to go
777 links to go
772 links to go
767 links to go
762 links to go
757 links to go
752 links to go
747 links to go
742 links to go
737 links to go
732 links to go
727 links to go
722 links to go
717 links to go
712 links to go
707 links to go
702 links to go
697 links to go
692 links to go
687 links to go
682 links to go
677 links to go
672 links to go
667 links to go
662 links to go
657 links to go
652 links to go
647 links to go
642 links to go
637 links to go
632 links to go
627 links to go
622 links to go
617 links to go
612 links to go
607 links to go
602 links to go
597 links to go
592 links to go
587 links to go
582 link

In [30]:
all_links = [i for j in data for i in j]

In [33]:
file_name = "all_links.pkl"

open_file = open(file_name, "wb")
pickle.dump(all_links, open_file)
open_file.close()

In [57]:
open_file = open(file_name, "rb")
game_links = pickle.load(open_file)
open_file.close()

In [60]:
all_links

['https://www.fotmob.com/match/3055941/matchfacts/liverpool-vs-sheffield-united',
 'https://www.fotmob.com/match/3066145/matchfacts/derby-county-vs-barnsley',
 'https://www.fotmob.com/match/3066153/matchfacts/swansea-city-vs-charlton-athletic',
 'https://www.fotmob.com/match/3093453/matchfacts/real-valladolid-vs-leganes',
 'https://www.fotmob.com/match/3093451/matchfacts/sevilla-vs-athletic-club',
 'https://www.fotmob.com/match/3093449/matchfacts/granada-vs-mallorca',
 'https://www.fotmob.com/match/3093450/matchfacts/real-sociedad-vs-villarreal',
 'https://www.fotmob.com/match/3093444/matchfacts/deportivo-alaves-vs-real-betis',
 'https://www.fotmob.com/match/3093446/matchfacts/celta-vigo-vs-osasuna',
 'https://www.fotmob.com/match/3068874/matchfacts/hibernian-vs-hearts',
 'https://www.fotmob.com/match/3056053/matchfacts/everton-vs-leicester-city',
 'https://www.fotmob.com/match/3241232/matchfacts/philadelphia-union-vs-la-galaxy',
 'https://www.fotmob.com/match/3241231/matchfacts/dc-uni

In [59]:
stats_data = []
for i in range(0, len(game_links), 3):
    chunk = game_links[i:i+5]
    with concurrent.futures.ThreadPoolExecutor() as executor:
        results = executor.map(get_stats, chunk)
    for result in results:
        stats_data.append(result)
    print(f'{len(game_links) - i} links to go')
    time.sleep(30)

4087 links to go
4084 links to go
4081 links to go
4078 links to go


KeyboardInterrupt: 

In [10]:
np.dot([[1,2,3],
       [1,2,3]],[1,2,3]) + [1,2]

array([15, 16])

In [13]:
np.array([1,2]) + [1,2]

array([2, 4])